# Богданов Александр Иванович, Б05-003

## Задача 6.2

В городе Н. правительство решило начать борьбу с превышениями скорости автомобилей. Для выбора стратегии борьбы оно сначала решило провести исследования касательно того, влияет ли используемый водителем автомобиль на среднюю скорость передвижения.

Для этого было сформировано 3 выборки по 20 человек, в каждой из которой людям выдали одинаковые автомобили марок Mitsubishi, Audi и BMW, соответственно. В течение месяца замерялась средняя скорость каждого из автомобилей.

Каждая из пар групп была проверена двувыборочным критерием на равенство распределений, также была проведена поправка на множественность гипотез.

Требуется:

1. Описать, в чём недостаток подхода правительства;
2. Предложить метод для более корректного решения задачи;
3. Записать формальное условие задачи.

## Решение|

In [37]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt

1. Плохо, что люди знают, что они участвуют в исследовании - это означает, что они могут вести себя не как обычно. Надо было, например, найти людей, которые ездят на данных марках автомобилей, и следить за ними, чтобы они ничего не знали (для чистоты эксперимента).

2. Использовать двувыборочные критерии, а потом делать поправку на множественность - плохо, так как решается более частная задача и непонятно, как определять размер эффекта для всех групп в совокупности.

3. Надо сосредоточиться на равенстве средних, а не распределений.

Надо использовать дисперсионный анализ, так как он позволит избежать множественное тестирование.

Предлагаю воспользоваться критерием Джонкхиера.

Пусть есть выборки $X_1, X_2, X_3$, также можно считать, что $X_k \sim F(x + \Delta_k)$.

Ставится гипотеза и альтернатива:
$$
\begin{equation*}
    \begin{cases}
        H_0: \  \Delta_1 = \Delta_2 = \Delta_3\ (\text{med}\ X_1 = \text{med}\ X_2 = \text{med}\ X_3) \\
        H_1: \  \text{med}\ X_1 \leq \text{med}\ X_2 \leq \text{med}\ X_3
    \end{cases}
\end{equation*}
$$

Статистика: $T(X) = \sum\limits_{k = 1}^3 \sum\limits_{i = 1}^{20} a_{ki}$, где $a_{ki}$ - число наблюдений из первых $k − 1$ выборок меньших, чем $X_{ki}$.

$T(X) \sim N(\mu, \sigma^2)$ (можно так аппроксимировать), где $\mu = 600$, $\sigma = 5433$.

Для проверки гипотезы на уровне значимости $\alpha$: $\tilde{T}_1 = N_{\frac{\alpha}{2}}(\mu, \sigma^2)$, $\tilde{T}_2 = N_{1 - \frac{\alpha}{2}}(\mu, \sigma^2)$:

$$
\begin{equation*}
    \begin{cases}
        \tilde{T}_1 < T < \tilde{T}_2\ \text{не отклоняем}\ H_0 \\
        T \leq \tilde{T}_1\ \text{или}\ \tilde{T}_2 \leq T\ \text{отклоняем}\ H_0
    \end{cases}
\end{equation*}
$$

In [15]:
mu = 1 / 4 * (60**2 - 3 * 20**2)
print(f"mu = {mu:.0f}")
sigma = 1 / 72 * (60**2 * (2 * 60 + 3) - 3 * 20**2 * (2 * 20 + 3))
print(f"sigma = {sigma:.0f}")

mu = 600
sigma = 5433


In [3]:
data = pd.read_csv('data/gov.csv')
data

,mitsubishi,audi,bmw
0,51,40,106
1,67,77,78
2,33,66,78
3,140,84,95
4,96,59,87
5,87,79,71
6,66,101,89
7,81,54,83
8,80,76,51
9,21,82,81


In [27]:
mitsubishi = data['mitsubishi'].to_numpy()
audi = data['audi'].to_numpy()
bmw = data['bmw'].to_numpy()

In [28]:
A = []
for car1 in mitsubishi:
    count = 0
    for car2 in audi:
        if car1 < car2:
            count += 1
    A.append(count)
    
B = []
for car1 in mitsubishi:
    count = 0
    for car2 in bmw:
        if car1 < car2:
            count += 1
    B.append(count)
    
C = []
for car1 in audi:
    count = 0
    for car2 in bmw:
        if car1 < car2:
            count += 1
    C.append(count)

In [32]:
A = np.array(A)
B = np.array(B)
C = np.array(C)

In [35]:
T = 2 * (A.sum() + B.sum() + C.sum()) - 3 * 20**2
T1 = st.norm(mu, sigma).ppf(0.05 / 2)
T2 = st.norm(mu, sigma).ppf(1 - 0.05 / 2)

In [36]:
print(f"T = {T}")
print(f"T1 = {T1}")
print(f"T2 = {T2}")

T = 220
T1 = -10049.137649334296
T2 = 11249.137649334292


Как мы видим, гипотеза не отклоняется. Следовательно нельзя считать, что средняя скорость автомобилей разных марок различна.